In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [3]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

In [4]:
def init_graph_attr(G, AdjFile, df, columns):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    capability_vector = list(zip(*(df[col] for col in columns)))
    node_attri_dict = dict(zip(df["District"], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["nodeStability"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

In [5]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1

def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [6]:
def get_node_stress(G,dim):
    stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return stress_dict

In [7]:
def get_node_stability(G,dim):
    stability_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStability"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stability_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStability"]
        except(KeyError):
            pass
    return stability_dict

In [8]:
G = nx.Graph()
df = pd.read_excel('KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.xlsx')
adjacency_file = 'Karnataka_District_Adjacency_File.xlsx'

# existing_data = pd.DataFrame(df['District'])

existing_data = pd.read_csv('result.csv')
# adjacency_file = "KAG 2016-17/Karnataka_District_Adjacency_File.xlsx"

In [9]:
def calculate_and_return_initial_stability(df, columns, G, adjacency_file = adjacency_file):
    dim = len(columns)
    col_to_pass = []

    for column in columns:
        column_values = df[column].values.reshape(-1, 1)
        scaler = MinMaxScaler()
        normalized_column_values = scaler.fit_transform(column_values)
        df[f"Normalized_{column}"] = normalized_column_values
        col_to_pass.append(f"Normalized_{column}")

    init_graph_attr(G, adjacency_file, df, col_to_pass)
    initial_stability = get_node_stability(G, dim)
    df["Initial Stability"] = df["District"].map(initial_stability)

    return df["Initial Stability"]

In [10]:
columns_to_normalize = ["Rice_Production_223", "Jowar_Yield_278"]
dim = 1
for col in columns_to_normalize:
    column_values = df[col].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    normalized_values = scaler.fit_transform(column_values)
    df[f"Normalized_{col}"] = normalized_values.flatten()
    init_graph_attr(G,adjacency_file , df,[f"Normalized_{col}"] )
    initial_stability = get_node_stability(G, dim)
    df[f"Initial Stability_{col}"] = df["District"].map(initial_stability)

In [11]:
# df["Initial Stability_Rice_Production_223"]

In [12]:
existing_data = pd.DataFrame(existing_data['District'])

In [13]:
# def calculate_impact_score(df, base_column, capability_vector, intervention):
#     # Perform simple linear regression
#     X = sm.add_constant(df[base_column])
#     y = df[capability_vector]
#     model = sm.OLS(y, X).fit()

#     # Get coefficients and intercept
#     m, c = model.params[base_column], model.params['const']
#     print(c)
#     # Predicted change in capability_vector for base_column + change_percentage% and base_column - change_percentage%
#     base_column_increase = (1 + intervention/100) * df[base_column]
#     base_column_decrease = (1 - intervention/100) * df[base_column]

#     rp_new_plus = m * base_column_increase + c 
#     rp_new_minus = m * base_column_decrease + c 

#     vector_plus = rp_new_plus - m * df[base_column] - c  
#     vector_minus = rp_new_minus - m * df[base_column] - c

#     # Generate normalized values using MinMaxScaler
#     scaler = MinMaxScaler()

#     normalized_vector_plus = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
#     normalized_vector_minus = scaler.fit_transform(vector_minus.values.reshape(-1, 1))

#     # Create new DataFrame with appropriate column names
#     result_df = pd.DataFrame({
#         f'{capability_vector} ({base_column} +{intervention}%)': rp_new_plus,
#         f'{capability_vector} ({base_column} -{intervention}%)': rp_new_minus,
#         f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_vector_plus.flatten(),
#         f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_vector_minus.flatten()
#     })

#     return result_df

In [14]:
# base_Column = "TotalNPK_315"
# CapabilityVector = "Jowar_Yield_278"
# change_percentage = 20

# result_df = calculate_impact_score(df, base_Column, CapabilityVector, change_percentage)

# result_df.head()

In [15]:
# def merge_columns_into_dataframe(result_df, existing_data):
#     for column in result_df.columns:
#         existing_data[column] = result_df[column]
#     return existing_data

# # Assuming existing_data is already defined
# existing_data = merge_columns_into_dataframe(result_df, existing_data)
# existing_data.head()

In [16]:
# result_df.head()

In [17]:
existing_data.head()

,District
0,BENGALURU
1,BENGALURU(R)
2,RAMANAGARA
3,CHITRADURGA
4,DAVANAGERE


In [18]:
dim = 1

In [19]:
def calculate_and_map_stability(G, existing_data, adjacency_file, columns_to_pass, inter, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stability = get_node_stability(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if inter >= 0 else '-'} {abs(inter)}%)"
    stability_column = existing_data["District"].map(NPK_stability)

    return stability_column, stability_column_name

## 1D Capability Vector

In [20]:
from bokeh.models import Span, Label
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

# Define the function to calculate and visualize impact stability
def calc_and_vis_impact_stability_1D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
    # Calculate Impact Score
    X = sm.add_constant(df[base_column])
    y = df[capability_vector]
    model = sm.OLS(y, X).fit()
    
    m, c = model.params[base_column], model.params['const']

    base_column_increase = (1 + intervention/100) * df[base_column]
    base_column_decrease = (1 - intervention/100) * df[base_column]

    new_vector_increase = m * base_column_increase + c
    new_vector_decrease = m * base_column_decrease + c
    
    vector_plus = new_vector_increase - m * df[base_column] - c  
    vector_minus = new_vector_decrease - m * df[base_column] - c

    scaler = MinMaxScaler()
    normalized_change_increase_vector = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
    normalized_change_decrease_vector = scaler.fit_transform(vector_minus.values.reshape(-1, 1))
    normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))
    normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))

    result_df = pd.DataFrame({
        f'{capability_vector} ({base_column} +{intervention}%)': new_vector_increase,
        f'{capability_vector} ({base_column} -{intervention}%)': new_vector_decrease,
        f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
        f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} +{intervention}%)': normalized_change_increase_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} -{intervention}%)': normalized_change_decrease_vector.flatten(),
    })
    
    # Merge Columns
    for column in result_df.columns:
        existing_data[column] = result_df[column]

    # Calculate Stability and add to DataFrame
    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    # Visualize with Plotly for +intervention
    fig_increase = px.scatter(existing_data,
                              x=stability_column_name,
                              y=f'Impact Score {capability_vector} ({base_column} +{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} +{intervention}%)',
                              labels={'x': f'Stability ({base_column} +{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)
    
    fig_increase.update_traces(hoverlabel=dict(bgcolor='grey', font=dict(color='white')))

    # Calculate average values from the DataFrame for +intervention
    avg_x_increase = existing_data[stability_column_name].mean()
    avg_y_increase = existing_data[f'Normalized {capability_vector} ({base_column} +{intervention}%)'].mean()

    # Add average lines for +intervention
    fig_increase.add_shape(
        type='line',
        x0=avg_x_increase,
        x1=avg_x_increase,
        y0=fig_increase.data[0].y.min(),
        y1=fig_increase.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_increase.add_shape(
        type='line',
        x0=fig_increase.data[0].x.min(),
        x1=fig_increase.data[0].x.max(),
        y0=avg_y_increase,
        y1=avg_y_increase,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for +intervention
    fig_increase.add_annotation(
        x=avg_x_increase,
        y=fig_increase.data[0].y.max(),
        text=f'Avg {stability_column_name}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_increase.add_annotation(
        x=fig_increase.data[0].x.max(),
        y=avg_y_increase,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for +intervention
    fig_increase.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} +{intervention}%): {x}"
                                          for x, y, d in zip(fig_increase.data[0].x, fig_increase.data[0].y, existing_data['District'])])

    # Visualize with Plotly for -intervention
    fig_decrease = px.scatter(existing_data,
                              x=stability_column_name_opposite,
                              y=f'Impact Score {capability_vector} ({base_column} -{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} -{intervention}%)',
                              labels={'x': f'Stability ({base_column} -{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)

    # Calculate average values from the DataFrame for -intervention
    avg_x_decrease = existing_data[stability_column_name_opposite].mean()
    avg_y_decrease = existing_data[f'Normalized {capability_vector} ({base_column} -{intervention}%)'].mean()

    # Add average lines for -intervention
    fig_decrease.add_shape(
        type='line',
        x0=avg_x_decrease,
        x1=avg_x_decrease,
        y0=fig_decrease.data[0].y.min(),
        y1=fig_decrease.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_decrease.add_shape(
        type='line',
        x0=fig_decrease.data[0].x.min(),
        x1=fig_decrease.data[0].x.max(),
        y0=avg_y_decrease,
        y1=avg_y_decrease,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for -intervention
    fig_decrease.add_annotation(
        x=avg_x_decrease,
        y=fig_decrease.data[0].y.max(),
        text=f'Avg {stability_column_name_opposite}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_decrease.add_annotation(
        x=fig_decrease.data[0].x.max(),
        y=avg_y_decrease,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for -intervention
    fig_decrease.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} -{intervention}%): {x}"
                                          for x, y, d in zip(fig_decrease.data[0].x, fig_decrease.data[0].y, existing_data['District'])])

    # Show the plots
    fig_increase.show()
    fig_decrease.show()

    return existing_data

base_Column = "TotalNPK_315"
CapabilityVector = "Rice_Production_223"
change_percentage = 20
inter = [10,20]
dim = 1
# Call the function and get the handles for the two plots
new_data_20 = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data_20.head()


,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.098721,0.901279,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.075418,0.924582,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.007616,0.992384,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.171050,0.828950,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.537816,0.462184,0.827968,0.827968


In [21]:
# ref = new_data[["District","Impact Score Rice_Production_223 (TotalNPK_315 +10%)","Impact Score Rice_Production_223 (TotalNPK_315 -10%)"]]

In [22]:
# cols_to_concat = ['Impact Score Rice_Production_223 (TotalNPK_315 +20%)', 'Impact Score Rice_Production_223 (TotalNPK_315 -20%)']
# res = pd.concat([ref, new_data_20[cols_to_concat]], axis=1)

In [23]:
# res.to_csv('res1.csv',index = False)

In [24]:
change_percentage = 10
dim = 1

# Call the function and get the handles for the two plots
new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data.head()

,District,Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),New Stability(NPK + 10%),New Stability(NPK - 10%)
0,BENGALURU,68479.019211,57507.638667,0.098721,0.098721,0.098721,0.901279,0.959717,0.959717
1,BENGALURU(R),57992.196208,48927.510755,0.075418,0.075418,0.075418,0.924582,0.973330,0.973330
2,RAMANAGARA,27479.191334,23962.324950,0.007616,0.007616,0.007616,0.992384,0.853066,0.853066
3,CHITRADURGA,101029.330849,84139.711825,0.171050,0.171050,0.171050,0.828950,0.733313,0.733313
4,DAVANAGERE,266084.831886,219185.121765,0.537816,0.537816,0.537816,0.462184,0.827968,0.827968


In [25]:

# base_Column = "TotalNPK_315"
# CapabilityVector = "Maize_Production_226"
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()

In [26]:
# Save the resulting DataFrame to a new CSV file if needed
# existing_data.to_csv('result.csv', index=False)

In [27]:
# from bokeh.models import Span, Label
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models.sources import ColumnDataSource
# from bokeh.models.tools import HoverTool
# import statsmodels.api as sm
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
#     # Calculate min and max values for changes made by each column in capability_vector
#     min_value_increase = float('inf')
#     max_value_increase = float('-inf')
#     min_value_decrease = float('inf')
#     max_value_decrease = float('-inf')
#     col_pass_plus = []
#     col_pass_minus = []
    
#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         change_vector_increase = m * base_column_increase + c
#         change_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Update min and max values for each column's changes
#         min_value_increase = min(min_value_increase, vector_plus.min())
#         max_value_increase = max(max_value_increase, vector_plus.max())

#         min_value_decrease = min(min_value_decrease, vector_minus.min())
#         max_value_decrease = max(max_value_decrease, vector_minus.max())

#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         new_vector_increase = m * base_column_increase + c
#         new_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Normalize changes manually using the calculated min and max values
#         scaler3 = MinMaxScaler()
#         normalized_new_increase_vector = scaler3.fit_transform(new_vector_increase.values.reshape(-1, 1))
    
#         scaler4 = MinMaxScaler()
#         normalized_new_decrease_vector = scaler4.fit_transform(new_vector_decrease.values.reshape(-1, 1))
        
#         normalized_change_increase_vector = (vector_plus - min_value_increase) / (max_value_increase - min_value_increase)
#         normalized_change_decrease_vector = (vector_minus - min_value_decrease) / (max_value_decrease - min_value_decrease)

#         result_df = pd.DataFrame({
#             f'{cap_col} ({base_column} +{intervention}%)': change_vector_increase,
#             f'{cap_col} ({base_column} -{intervention}%)': change_vector_decrease,
#             f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
#             f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
#             f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
#             f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
#         })

#         # Merge Columns
#         for column in result_df.columns:
#             if column in result_df.columns:
#                 existing_data[column] = result_df[column]

#         # Calculate Stability and add to DataFrame
#         columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
#         col_pass_plus.append(columns_to_pass)

#         # Calculate Stability for the opposite intervention and add to DataFrame
#         opposite_intervention = -intervention
#         columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
#         col_pass_minus.append(columns_to_pass_opposite)
        
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

#     existing_data[stability_column_name] = stability_column

#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

#     existing_data[stability_column_name_opposite] = stability_column_opposite
        

# # Visualize with Plotly for +intervention
#     fig_increase = go.Figure()
#     color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for +intervention
#         scatter_increase = go.Scatter(
#             x=existing_data[f"{stability_column_name}"],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=8, color=color_palette[i]),
#             name=f'{cap_col} +{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_increase.add_trace(scatter_increase)

#     # Update layout for +intervention plot
#     fig_increase.update_layout(
#         title=f'Impact vs Stability (+{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Visualize with Plotly for -intervention
#     fig_decrease = go.Figure()

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for -intervention
#         scatter_decrease = go.Scatter(
#             x=existing_data[stability_column_name_opposite],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=8, color=color_palette[i]),
#             name=f'{cap_col} -{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_decrease.add_trace(scatter_decrease)

#     # Update layout for -intervention plot
#     fig_decrease.update_layout(
#         title=f'Impact vs Stability (-{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Show plots
#     fig_increase.show()
#     fig_decrease.show()

#     return existing_data, fig_increase, fig_decrease

# # Assuming df is your DataFrame
# base_column = "TotalNPK_315"
# capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data, handle_increase, handle_decrease = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()


In [28]:
# new_data.head()

In [29]:
# new_data.to_csv('test.csv', index=False)

## 2D Capability vector

In [30]:
res_df = pd.DataFrame()

In [87]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import cycle

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    # Calculate min and max values for changes made by each column in capability_vector
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    for intervention in change_percentages:
        for cap_col in capability_vector:
                # Calculate Impact Score
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']

            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            # Update min and max values for each column's changes (These are the new values for each column after the intervention)
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
            vector_plus = change_vector_increase - m * df[base_column] - c  
            vector_minus = change_vector_decrease - m * df[base_column] - c
            
            # Update min and max values for each column's changes
            min_value = min(min_value, vector_plus.min(), vector_minus.min())
            max_value = max(max_value, vector_plus.max(), vector_minus.max())


    print(f"min_value: {min_value}")
    print(f"max_value: {max_value}")
    print(f"changed_min_value: {change_min_value}")
    print(f"changed_max_value: {change_max_value}")    # Normalize changes manually using the calculated min and max values
    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            # Calculate Impact Score
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            vector_plus = new_vector_increase - m * df[base_column] - c  
            vector_minus = new_vector_decrease - m * df[base_column] - c
            
            normalized_change_increase_vector = (vector_plus - min_value) / (max_value - min_value)
            normalized_change_decrease_vector = (vector_minus - min_value) / (max_value - min_value)
            
            # scaler = MinMaxScaler()
            # normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))      
            # normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))  
            
            # new_vector_increase_val = new_vector_increase.to_numpy().reshape(-1, 1)
            # normalized_new_increase_vector = (new_vector_increase_val - new_vector_increase_val.min()) / (new_vector_increase_val.max() - new_vector_increase_val.min())

            # # For new_vector_decrease
            # new_vector_decrease_val = new_vector_decrease.to_numpy().reshape(-1, 1)
            # normalized_new_decrease_vector = (new_vector_decrease_val - new_vector_decrease_val.min()) / (new_vector_decrease_val.max() - new_vector_decrease_val.min())
                        
            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
                    f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
                })
            ], axis=1)

            result_df_1 = pd.DataFrame({
                f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                # f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                # f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
                # f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
                # f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
            })

            # Merge Columns
            
                
            for column in result_df_1.columns:
                res_df[column] = result_df_1[column]
            # Calculate Stability and add to DataFrame
            columns_to_pass = f'Normalized {cap_col} ({base_column} +{intervention}%)'
            # columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'Normalized {cap_col} ({base_column} -{intervention}%)'
            # columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]
    # print(existing_data.columns)
    
    # for intervention in change_percentages:
        # Normalize stability values
        # print(col_pass_minus)
        # print(col_pass_plus)
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
        
    # Define a color palette
    color_palette = ['red', 'blue', 'green', 'orange', 'purple', 'violet', 'yellow']

    # Create a color cycle iterator
    color_cycle = cycle(color_palette)

    # Create a dictionary to map capability vectors to colors
    capability_color_map = {cap_col: next(color_cycle) for cap_col in capability_vector}

    fig = make_subplots(rows=len(change_percentages), cols=2, shared_xaxes=True, shared_yaxes=True,vertical_spacing=0.1, horizontal_spacing=0.1, row_heights=[0.5] * len(change_percentages))

    for idx, intervention in enumerate(change_percentages):
        for cap_col in capability_vector:
            color = capability_color_map[cap_col]

            fig.update_xaxes(title_text=f'Stability ({base_column} +{intervention}%)', row=idx + 1, col=1)
            fig.update_yaxes(title_text='Impact', row=idx + 1, col=1)

            # Visualize with Plotly for +intervention
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Impact Score {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )

            fig.add_trace(scatter_increase, row=idx + 1, col=1)

            # Add average lines
            avg_stability_increase = existing_data[stability_column_name_modified].mean()
            avg_impact_increase = existing_data[
                f"Impact Score {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

            fig.add_shape(
                go.layout.Shape(
                    type="line",
                    x0=avg_stability_increase,
                    x1=avg_stability_increase,
                    y0=0,
                    y1=1,
                    line=dict(color="black", dash="dash"),
                    name="Average Stability"
                ),
                row=idx + 1, col=1
            )
            fig.add_shape(
                go.layout.Shape(
                    type="line",
                    x0=0,
                    x1=1,
                    y0=avg_impact_increase,
                    y1=avg_impact_increase,
                    line=dict(color="black", dash="dash"),
                    name="Average Impact"
                ),
                row=idx + 1, col=1
            )

            # Update layout for -intervention plot
            fig.update_xaxes(title_text=f'Stability ({base_column} -{intervention}%)', row=idx + 1, col=2)

            # Visualize with Plotly for -intervention
            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[
                    f"Impact Score {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )

            fig.add_trace(scatter_decrease, row=idx + 1, col=2)

            # Add average lines for -intervention
            avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
            avg_impact_decrease = existing_data[
                f"Impact Score {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

            fig.add_shape(
                go.layout.Shape(
                    type="line",
                    x0=avg_stability_decrease,
                    x1=avg_stability_decrease,
                    y0=0,
                    y1=1,
                    line=dict(color="black", dash="dash"),
                    name="Average Stability"
                ),
                row=idx + 1, col=2
            )
            fig.add_shape(
                go.layout.Shape(
                    type="line",
                    x0=0,
                    x1=1,
                    y0=avg_impact_decrease,
                    y1=avg_impact_decrease,
                    line=dict(color="black", dash="dash"),
                    name="Average Impact"
                ),
                row=idx + 1, col=2
            )

    # Update subplot layout
    fig.update_layout(
        height=800,  
        showlegend=True,
        legend=dict(title="Capability", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    # Show plots
    fig.show()


    return existing_data, result_df

base_column = "TotalNPK_315"
capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

min_value: -84717.31990045196
max_value: 84717.31990045196
changed_min_value: 412.79251570427334
changed_max_value: 516440.3456216234


,District,Initial Stability,Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),Jowar_Yield_278 (TotalNPK_315 +10%),Jowar_Yield_278 (TotalNPK_315 -10%),...,Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Rice_Jowar,New Stability(NPK - 20%)_Rice_Jowar
0,BENGALURU,0.313550,68479.019211,57507.638667,0.131904,0.110643,0.532376,0.467624,616.168067,570.414310,...,0.564753,0.435247,639.044946,547.537431,0.000438,0.000261,0.500270,0.499730,0.959044,0.959044
1,BENGALURU(R),0.742284,57992.196208,48927.510755,0.111582,0.094016,0.526750,0.473250,572.435052,534.632751,...,0.553500,0.446500,591.336202,515.731601,0.000346,0.000199,0.500223,0.499777,0.976505,0.976505
2,RAMANAGARA,0.608105,27479.191334,23962.324950,0.052451,0.045636,0.510378,0.489622,445.187197,430.520870,...,0.520756,0.479244,452.520360,423.187707,0.000077,0.000020,0.500087,0.499913,0.822250,0.822250
3,CHITRADURGA,0.422882,101029.330849,84139.711825,0.194983,0.162253,0.549841,0.450159,751.912067,681.477582,...,0.599682,0.400318,787.129310,646.260340,0.000725,0.000452,0.500416,0.499584,0.668650,0.668650
4,DAVANAGERE,0.339007,266084.831886,219185.121765,0.514841,0.423955,0.638401,0.361599,1440.240158,1244.655111,...,0.776801,0.223199,1538.032682,1146.862588,0.002181,0.001423,0.501154,0.498846,0.790058,0.790058


In [73]:
new_data_1

,District,Initial Stability,Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),Jowar_Yield_278 (TotalNPK_315 +10%),Jowar_Yield_278 (TotalNPK_315 -10%),...,Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Rice_Jowar,New Stability(NPK - 20%)_Rice_Jowar
0,BENGALURU,0.313550,68479.019211,57507.638667,0.131904,0.110643,0.532376,0.467624,616.168067,570.414310,...,0.564753,0.435247,639.044946,547.537431,0.000438,0.000261,0.500270,0.499730,0.959044,0.959044
1,BENGALURU(R),0.742284,57992.196208,48927.510755,0.111582,0.094016,0.526750,0.473250,572.435052,534.632751,...,0.553500,0.446500,591.336202,515.731601,0.000346,0.000199,0.500223,0.499777,0.976505,0.976505
2,RAMANAGARA,0.608105,27479.191334,23962.324950,0.052451,0.045636,0.510378,0.489622,445.187197,430.520870,...,0.520756,0.479244,452.520360,423.187707,0.000077,0.000020,0.500087,0.499913,0.822250,0.822250
3,CHITRADURGA,0.422882,101029.330849,84139.711825,0.194983,0.162253,0.549841,0.450159,751.912067,681.477582,...,0.599682,0.400318,787.129310,646.260340,0.000725,0.000452,0.500416,0.499584,0.668650,0.668650
4,DAVANAGERE,0.339007,266084.831886,219185.121765,0.514841,0.423955,0.638401,0.361599,1440.240158,1244.655111,...,0.776801,0.223199,1538.032682,1146.862588,0.002181,0.001423,0.501154,0.498846,0.790058,0.790058
5,KOLAR,0.771152,65579.675404,55135.448279,0.126286,0.106046,0.530821,0.469179,604.076984,560.521605,...,0.561642,0.438358,625.854674,538.743916,0.000413,0.000244,0.500257,0.499743,1.000000,1.000000
6,CHIKKABALLAPURA,0.799368,81544.187243,68197.321602,0.157223,0.131358,0.539386,0.460614,670.653511,614.993309,...,0.578773,0.421227,698.483612,587.163208,0.000554,0.000338,0.500329,0.499671,0.997250,0.997250
7,SHIVAMOGGA,0.279347,154680.134786,128035.824137,0.298952,0.247318,0.578627,0.421373,975.651084,864.536777,...,0.657254,0.342746,1031.208237,808.979624,0.001198,0.000768,0.500656,0.499344,0.967812,0.967812
8,TUMAKURU,0.626449,106890.147260,88934.925252,0.206340,0.171545,0.552986,0.447014,776.353328,701.474977,...,0.605971,0.394029,813.792503,664.035802,0.000777,0.000487,0.500442,0.499558,0.988802,0.988802
9,CHIKKAMAGALURU,0.818925,183813.362651,151872.101481,0.355408,0.293510,0.594258,0.405742,1097.144878,963.940791,...,0.688517,0.311483,1163.746922,897.338747,0.001455,0.000939,0.500786,0.499214,0.850936,0.850936


In [68]:
base_column = "TotalNPK_315"
capability_vector = ["Rice_Production_223", "Maize_Production_226"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

,District,Initial Stability,Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),Maize_Production_226 (TotalNPK_315 +10%),Maize_Production_226 (TotalNPK_315 -10%),...,Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Maize_Production_226 (TotalNPK_315 +20%),Maize_Production_226 (TotalNPK_315 -20%),Normalized Maize_Production_226 (TotalNPK_315 +20%),Normalized Maize_Production_226 (TotalNPK_315 -20%),Impact Score Maize_Production_226 (TotalNPK_315 +20%),Impact Score Maize_Production_226 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.973840,68479.019211,57507.638667,0.127026,0.099468,0.529708,0.470292,66324.632613,54367.859761,...,0.559416,0.440584,72303.019039,48389.473335,0.136631,0.076565,0.564753,0.435247,0.959044,0.959044
1,BENGALURU(R),0.983965,57992.196208,48927.510755,0.100685,0.077917,0.524545,0.475455,54895.937738,45017.109409,...,0.549091,0.450909,59835.351903,40077.695244,0.105315,0.055688,0.553500,0.446500,0.976505,0.976505
2,RAMANAGARA,0.868345,27479.191334,23962.324950,0.024043,0.015209,0.509523,0.490477,21642.413549,17809.680527,...,0.519046,0.480954,23558.780060,15893.314016,0.014196,-0.005058,0.520756,0.479244,0.822250,0.822250
3,CHITRADURGA,0.477695,101029.330849,84139.711825,0.208785,0.166362,0.545734,0.454266,101798.443858,83391.887143,...,0.591467,0.408533,111001.722216,74188.608786,0.233834,0.141367,0.599682,0.400318,0.668650,0.668650
4,DAVANAGERE,0.169111,266084.831886,219185.121765,0.623369,0.505567,0.626995,0.373005,281678.371090,230566.373060,...,0.753989,0.246011,307234.370105,205010.374046,0.726728,0.469963,0.776801,0.223199,0.790058,0.790058


In [84]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    # Calculate min and max values for changes made by each column in capability_vector
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    for intervention in change_percentages:
        for cap_col in capability_vector:
                # Calculate Impact Score
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']

            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            # Update min and max values for each column's changes (These are the new values for each column after the intervention)
            change_min_value = min(change_min_value, change_vector_increase.min())
            change_max_value = max(change_max_value, change_vector_decrease.max())

            change_min_value = min(change_min_value, change_vector_increase.min())
            change_max_value = max(change_max_value, change_vector_decrease.max())
            
            vector_plus = change_vector_increase - m * df[base_column] - c  
            vector_minus = change_vector_decrease - m * df[base_column] - c
            
            # Update min and max values for each column's changes
            min_value = min(min_value, vector_plus.min())
            max_value = max(max_value, vector_plus.max())

            min_value = min(min_value, vector_minus.min())
            max_value = max(max_value, vector_minus.max())


    # print(f"min_value: {min_value}")
    # print(f"max_value: {max_value}")
    # print(f"change_min_value: {change_min_value}")
    # print(f"change_max_value: {change_max_value}")    # Normalize changes manually using the calculated min and max values
    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            # Calculate Impact Score
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            vector_plus = new_vector_increase - m * df[base_column] - c  
            vector_minus = new_vector_decrease - m * df[base_column] - c

            # Normalize changes manually using the calculated min and max values
            normalized_change_increase_vector = (vector_plus - min_value) / (max_value - min_value)
            normalized_change_decrease_vector = (vector_minus - min_value) / (max_value - min_value)
            
            # scaler = MinMaxScaler()
            # normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))      
            # normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))  
            
            # new_vector_increase_val = new_vector_increase.to_numpy().reshape(-1, 1)
            # normalized_new_increase_vector = (new_vector_increase_val - new_vector_increase_val.min()) / (new_vector_increase_val.max() - new_vector_increase_val.min())

            # # For new_vector_decrease
            # new_vector_decrease_val = new_vector_decrease.to_numpy().reshape(-1, 1)
            # normalized_new_decrease_vector = (new_vector_decrease_val - new_vector_decrease_val.min()) / (new_vector_decrease_val.max() - new_vector_decrease_val.min())
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
                    f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
                })
            ], axis=1)

            result_df_1 = pd.DataFrame({
                f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                # f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                # f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
                # f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
                # f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
            })

            # Merge Columns
            
                
            for column in result_df_1.columns:
                res_df[column] = result_df_1[column]
            # Calculate Stability and add to DataFrame
            columns_to_pass = f'Normalized {cap_col} ({base_column} +{intervention}%)'
            # columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'Normalized {cap_col} ({base_column} -{intervention}%)'
            # columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]
    # print(existing_data.columns)
    
    # for intervention in change_percentages:
        # Normalize stability values
        # print(col_pass_minus)
        # print(col_pass_plus)
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
        
        # Visualize with Plotly for +intervention
        fig_increase = go.Figure()
        color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed
        # print(existing_data.columns)

        for i, cap_col in enumerate(capability_vector):
            symbol = 'star' if i == 0 else 'circle'  # Use 'star' for the first point, 'circle' for others
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Impact Score {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7, symbol=symbol, color=color_palette[i]),  # Adjust the size and add symbol parameter
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            fig_increase.add_trace(scatter_increase)
        
        # Add average lines
        # print("aefawefaw")
        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_impact_increase = existing_data[f"Impact Score {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=0,
                y1=1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )
        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for +intervention plot
        fig_increase.update_layout(
            title=f'Impact vs Stability (+{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
            yaxis=dict(title='Normalized Impact'),
            hovermode='closest'
        )

        # Visualize with Plotly for -intervention
        fig_decrease = go.Figure()

        for i, cap_col in enumerate(capability_vector):
            # Scatter plot for -intervention
            symbol = 'star' if i == 0 else 'circle'
            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[f"Impact Score {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7,symbol=symbol, color=color_palette[i]),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )

            fig_decrease.add_trace(scatter_decrease)

        # Add average lines for -intervention
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        avg_impact_decrease = existing_data[f"Impact Score {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=0,
                y1=1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )
        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for -intervention plot
        fig_decrease.update_layout(
            title=f'Impact vs Stability (-{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
            yaxis=dict(title='Normalized Impact'),
            hovermode='closest'
        )

        # Show plots
        fig_increase.show()
        fig_decrease.show()

    return existing_data, result_df

base_column = "TotalNPK_315"
capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

,District,Initial Stability,Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),Jowar_Yield_278 (TotalNPK_315 +10%),Jowar_Yield_278 (TotalNPK_315 -10%),...,Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Rice_Jowar,New Stability(NPK - 20%)_Rice_Jowar
0,BENGALURU,0.313550,68479.019211,57507.638667,0.174961,0.146752,0.532376,0.467624,616.168067,570.414310,...,0.564753,0.435247,639.044946,547.537431,0.000535,0.000300,0.500270,0.499730,0.959044,0.959044
1,BENGALURU(R),0.742284,57992.196208,48927.510755,0.147998,0.124691,0.526750,0.473250,572.435052,534.632751,...,0.553500,0.446500,591.336202,515.731601,0.000413,0.000218,0.500223,0.499777,0.976505,0.976505
2,RAMANAGARA,0.608105,27479.191334,23962.324950,0.069545,0.060502,0.510378,0.489622,445.187197,430.520870,...,0.520756,0.479244,452.520360,423.187707,0.000056,-0.000020,0.500087,0.499913,0.822250,0.822250
3,CHITRADURGA,0.422882,101029.330849,84139.711825,0.258652,0.215227,0.549841,0.450159,751.912067,681.477582,...,0.599682,0.400318,787.129310,646.260340,0.000916,0.000554,0.500416,0.499584,0.668650,0.668650
4,DAVANAGERE,0.339007,266084.831886,219185.121765,0.683032,0.562446,0.638401,0.361599,1440.240158,1244.655111,...,0.776801,0.223199,1538.032682,1146.862588,0.002847,0.001841,0.501154,0.498846,0.790058,0.790058


In [ ]:
        # fig = make_subplots(rows=1, cols=len(capability_vector), subplot_titles=[f'{cap_col} ({base_column} {intervention:+}%)' for cap_col in capability_vector])

        # for j, cap_col in enumerate(capability_vector):
        #     # Scatter plot for +intervention
        #     scatter_increase = go.Scatter(
        #         x=existing_data[stability_column_name_modified],
        #         y=existing_data[f"Impact Score {cap_col} ({base_column} +{intervention}%)"],
        #         mode='markers',
        #         marker=dict(size=10, color='red'),  # Adjust color if needed
        #         name=f'{cap_col} +{intervention}%',
        #         text=existing_data['District'],
        #         hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
        #     )

        #     # Scatter plot for -intervention
        #     scatter_decrease = go.Scatter(
        #         x=existing_data[stability_column_name_opposite_modified],
        #         y=existing_data[f"Impact Score {cap_col} ({base_column} -{intervention}%)"],
        #         mode='markers',
        #         marker=dict(size=10, color='blue'),  # Adjust color if needed
        #         name=f'{cap_col} -{intervention}%',
        #         text=existing_data['District'],
        #         hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
        #     )

        #     # Add subplots
        #     fig.add_trace(scatter_increase, row=1, col=j+1)
        #     fig.add_trace(scatter_decrease, row=1, col=j+1)

        # # Update layout for each intervention plot
        # fig.update_layout(
        #     title=f'Impact vs Stability for {intervention}%',
        #     xaxis=dict(title=f'Stability ({base_column} {intervention}%)'),
        #     yaxis=dict(title='Impact'),
        #     showlegend=False
        # )

        # # Show the plot
        # fig.show()

In [32]:
capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
change_percentage = 10
dim = len(capability_vector)

new_data_1, handle_increase, handle_decrease,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data_1.head()

['Normalized Rice_Production_223 (TotalNPK_315 -10%)', 'Normalized Jowar_Yield_278 (TotalNPK_315 -10%)']
['Normalized Rice_Production_223 (TotalNPK_315 +10%)', 'Normalized Jowar_Yield_278 (TotalNPK_315 +10%)']


,District,Initial Stability,Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),Jowar_Yield_278 (TotalNPK_315 +10%),Jowar_Yield_278 (TotalNPK_315 -10%),Normalized Jowar_Yield_278 (TotalNPK_315 +10%),Normalized Jowar_Yield_278 (TotalNPK_315 -10%),Impact Score Jowar_Yield_278 (TotalNPK_315 +10%),Impact Score Jowar_Yield_278 (TotalNPK_315 -10%),New Stability(NPK + 10%)_Rice_Jowar,New Stability(NPK - 10%)_Rice_Jowar
0,BENGALURU,0.313550,68479.019211,57507.638667,0.143667,0.146778,0.129382,0.870618,616.168067,570.414310,0.000391,0.000390,0.000398,0.999602,0.959044,0.959044
1,BENGALURU(R),0.742284,57992.196208,48927.510755,0.121527,0.124718,0.106872,0.893128,572.435052,534.632751,0.000299,0.000298,0.000304,0.999696,0.976505,0.976505
2,RAMANAGARA,0.608105,27479.191334,23962.324950,0.057106,0.060532,0.041376,0.958624,445.187197,430.520870,0.000030,0.000030,0.000031,0.999969,0.822250,0.822250
3,CHITRADURGA,0.422882,101029.330849,84139.711825,0.212389,0.215251,0.199250,0.800750,751.912067,681.477582,0.000678,0.000675,0.000689,0.999311,0.668650,0.668650
4,DAVANAGERE,0.339007,266084.831886,219185.121765,0.560865,0.562460,0.553539,0.446461,1440.240158,1244.655111,0.002131,0.002123,0.002167,0.997833,0.790058,0.790058


In [28]:
# import statsmodels.api as sm
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def calc_and_vis_impact_stability(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
#     # Calculate min and max values for changes made by each column in capability_vector
#     min_value_increase = float('inf')
#     max_value_increase = float('-inf')
#     min_value_decrease = float('inf')
#     max_value_decrease = float('-inf')

#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         change_vector_increase = m * base_column_increase + c
#         change_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Update min and max values for each column's changes
#         min_value_increase = min(min_value_increase, vector_plus.min())
#         max_value_increase = max(max_value_increase, vector_plus.max())

#         min_value_decrease = min(min_value_decrease, vector_minus.min())
#         max_value_decrease = max(max_value_decrease, vector_minus.max())

#     # Normalize changes manually using the calculated min and max values
#     col_pass_plus = []
#     col_pass_minus = []
#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         new_vector_increase = m * base_column_increase + c
#         new_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = new_vector_increase - m * df[base_column] - c  
#         vector_minus = new_vector_decrease - m * df[base_column] - c

#         # Normalize changes manually using the calculated min and max values
#         normalized_change_increase_vector = (vector_plus - min_value_increase) / (max_value_increase - min_value_increase)
#         normalized_change_decrease_vector = (vector_minus - min_value_decrease) / (max_value_decrease - min_value_decrease)

#         scaler = MinMaxScaler()
#         normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))
#         normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))
        
#         result_df = pd.DataFrame({
#             f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
#             f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
#             f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
#             f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
#             f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
#             f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,

#         })

#         # Merge Columns
#         for column in result_df.columns:
#             if column in result_df.columns:
#                 existing_data[column] = result_df[column]

#         # Calculate Stability and add to DataFrame
#         columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
#         col_pass_plus.append(columns_to_pass)
        
#         # Calculate Stability for the opposite intervention and add to DataFrame
#         opposite_intervention = -intervention
#         columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
#         col_pass_minus.append(columns_to_pass_opposite)
        
#     # Normalize stability values
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

#     normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())
        
#     capability_first_words = [col.split('_')[0] for col in capability_vector]
#     stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
#     existing_data[stability_column_name_modified] = normalized_stability_vector
    
#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)
        
#     # Normalize stability values for the opposite intervention
#     normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
#     stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
#     existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
    
#     # Visualize with Plotly for +intervention
#     fig_increase = go.Figure()
#     color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed

#     for i, cap_col in enumerate(capability_vector):
#         symbol = 'star' if i == 0 else 'circle'  # Use 'star' for the first point, 'circle' for others
#         scatter_increase = go.Scatter(
#             x=existing_data[stability_column_name_modified],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=7, symbol=symbol, color=color_palette[i]),  # Adjust the size and add symbol parameter
#             name=f'{cap_col} +{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )
#         fig_increase.add_trace(scatter_increase)

#     # Add average lines
#     avg_stability_increase = existing_data[stability_column_name_modified].mean()
#     avg_impact_increase = existing_data[f"Normalized {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

#     fig_increase.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=avg_stability_increase,
#             x1=avg_stability_increase,
#             y0=0,
#             y1=1,
#             line=dict(color="black", dash="dash"),
#             name="Average Stability"
#         )
#     )
#     fig_increase.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=0,
#             x1=1,
#             y0=avg_impact_increase,
#             y1=avg_impact_increase,
#             line=dict(color="black", dash="dash"),
#             name="Average Impact"
#         )
#     )

#     # Update layout for +intervention plot
#     fig_increase.update_layout(
#         title=f'Impact vs Stability (+{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Visualize with Plotly for -intervention
#     fig_decrease = go.Figure()

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for -intervention
#         symbol = 'star' if i == 0 else 'circle'
#         scatter_decrease = go.Scatter(
#             x=existing_data[stability_column_name_opposite_modified],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=7,symbol=symbol, color=color_palette[i]),
#             name=f'{cap_col} -{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_decrease.add_trace(scatter_decrease)

#     # Add average lines for -intervention
#     avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
#     avg_impact_decrease = existing_data[f"Normalized {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

#     fig_decrease.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=avg_stability_decrease,
#             x1=avg_stability_decrease,
#             y0=0,
#             y1=1,
#             line=dict(color="black", dash="dash"),
#             name="Average Stability"
#         )
#     )
#     fig_decrease.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=0,
#             x1=1,
#             y0=avg_impact_decrease,
#             y1=avg_impact_decrease,
#             line=dict(color="black", dash="dash"),
#             name="Average Impact"
#         )
#     )

#     # Update layout for -intervention plot
#     fig_decrease.update_layout(
#         title=f'Impact vs Stability (-{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Show plots
#     fig_increase.show()
#     fig_decrease.show()

#     return existing_data, fig_increase, fig_decrease

# base_column = "TotalNPK_315"
# capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
# change_percentage = 20
# dim = 2

# new_data, handle_increase, handle_decrease = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
# new_data.head()

In [29]:
print(res_df['Rice_Production_223 (TotalNPK_315 +20%)'].min(), res_df['Rice_Production_223 (TotalNPK_315 +20%)'].max())

25498.600629373545 516440.3456216234


In [30]:
print(res_df['Jowar_Yield_278 (TotalNPK_315 +20%)'].min(), res_df['Jowar_Yield_278 (TotalNPK_315 +20%)'].max())

436.9275738990035 2484.293411637132


In [31]:
base_column = "TotalNPK_315"
capability_vector = ["Maize_Production_226", "Rice_Production_223"]
change_percentage = 20
dim = 1

new_data, handle_increase, handle_decrease,result_df = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data.head()

NameError: name 'calc_and_vis_impact_stability' is not defined

In [ ]:
base_column = "TotalNPK_315"
capability_vector = ["Maize_Production_226", "Rice_Production_223"]
change_percentage = 10
dim = 2

new_data, handle_increase, handle_decrease,result_df = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data.head()

2.564730341506419 562.38192631683
2.353363480056113 8136.426218911565


,District,Initial Stability,Maize_Production_226 (TotalNPK_315 +10%),Maize_Production_226 (TotalNPK_315 -10%),Normalized Maize_Production_226 (TotalNPK_315 +10%),Normalized Maize_Production_226 (TotalNPK_315 -10%),Impact Score Maize_Production_226 (TotalNPK_315 +10%),Impact Score Maize_Production_226 (TotalNPK_315 -10%),Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),New Stability(NPK + 10%)_Maize_Rice,New Stability(NPK - 10%)_Maize_Rice
0,BENGALURU,0.973840,66324.632613,54367.859761,0.098721,0.098721,0.101340,0.898660,68479.019211,57507.638667,0.098721,0.098721,0.090322,0.909678,0.959044,0.959044
1,BENGALURU(R),0.983965,54895.937738,45017.109409,0.075418,0.075418,0.078105,0.921895,57992.196208,48927.510755,0.075418,0.075418,0.069002,0.930998,0.976505,0.976505
2,RAMANAGARA,0.868345,21642.413549,17809.680527,0.007616,0.007616,0.010500,0.989500,27479.191334,23962.324950,0.007616,0.007616,0.006968,0.993032,0.822250,0.822250
3,CHITRADURGA,0.477695,101798.443858,83391.887143,0.171050,0.171050,0.173459,0.826541,101029.330849,84139.711825,0.171050,0.171050,0.156497,0.843503,0.668650,0.668650
4,DAVANAGERE,0.169111,281678.371090,230566.373060,0.537816,0.537816,0.539159,0.460841,266084.831886,219185.121765,0.537816,0.537816,0.492058,0.507942,0.790058,0.790058


In [ ]:
base_column = "TotalNPK_315"
capability_vector = ["Maize_Production_226", "Rice_Production_223","Jowar_Yield_278"]
change_percentage = 20
dim = 3

new_data_3d, handle_increase, handle_decrease,result_df = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data_3d.head()

2.564730341506419 562.38192631683
2.353363480056113 8136.426218911565
0.009814190872938423 364.522399314813


,District,Initial Stability,Maize_Production_226 (TotalNPK_315 +20%),Maize_Production_226 (TotalNPK_315 -20%),Normalized Maize_Production_226 (TotalNPK_315 +20%),Normalized Maize_Production_226 (TotalNPK_315 -20%),Impact Score Maize_Production_226 (TotalNPK_315 +20%),Impact Score Maize_Production_226 (TotalNPK_315 -20%),Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),...,Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Maize_Rice_Jowar,New Stability(NPK - 20%)_Maize_Rice_Jowar
0,BENGALURU,0.313061,72303.019039,48389.473335,0.098721,0.098721,0.129392,0.870608,73964.709483,52021.948395,...,0.118718,0.881282,639.044946,547.537431,0.098721,0.098721,0.000365,0.999635,0.959044,0.959044
1,BENGALURU(R),0.742141,59835.351903,40077.695244,0.075418,0.075418,0.106882,0.893118,62524.538934,44395.168029,...,0.098063,0.901937,591.336202,515.731601,0.075418,0.075418,0.000279,0.999721,0.976505,0.976505
2,RAMANAGARA,0.600687,23558.780060,15893.314016,0.007616,0.007616,0.041388,0.958612,29237.624526,22203.891757,...,0.037966,0.962034,452.520360,423.187707,0.007616,0.007616,0.000028,0.999972,0.822250,0.822250
3,CHITRADURGA,0.419056,111001.722216,74188.608786,0.171050,0.171050,0.199260,0.800740,109474.140361,75694.902314,...,0.182827,0.817173,787.129310,646.260340,0.171050,0.171050,0.000632,0.999368,0.668650,0.668650
4,DAVANAGERE,0.106020,307234.370105,205010.374046,0.537816,0.537816,0.553544,0.446456,289534.686947,195735.266704,...,0.507914,0.492086,1538.032682,1146.862588,0.537816,0.537816,0.001988,0.998012,0.790058,0.790058


In [ ]:
base_column = "TotalNPK_315"
capability_vector = ["Maize_Production_226", "Rice_Production_223","Jowar_Yield_278"]
change_percentage = 10
dim = 3

new_data_3d, handle_increase, handle_decrease, result_df = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data_3d.head()

2.564730341506419 562.38192631683
2.353363480056113 8136.426218911565
0.009814190872938423 364.522399314813


,District,Initial Stability,Maize_Production_226 (TotalNPK_315 +10%),Maize_Production_226 (TotalNPK_315 -10%),Normalized Maize_Production_226 (TotalNPK_315 +10%),Normalized Maize_Production_226 (TotalNPK_315 -10%),Impact Score Maize_Production_226 (TotalNPK_315 +10%),Impact Score Maize_Production_226 (TotalNPK_315 -10%),Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),...,Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),Jowar_Yield_278 (TotalNPK_315 +10%),Jowar_Yield_278 (TotalNPK_315 -10%),Normalized Jowar_Yield_278 (TotalNPK_315 +10%),Normalized Jowar_Yield_278 (TotalNPK_315 -10%),Impact Score Jowar_Yield_278 (TotalNPK_315 +10%),Impact Score Jowar_Yield_278 (TotalNPK_315 -10%),New Stability(NPK + 10%)_Maize_Rice_Jowar,New Stability(NPK - 10%)_Maize_Rice_Jowar
0,BENGALURU,0.313061,66324.632613,54367.859761,0.098721,0.098721,0.129392,0.870608,68479.019211,57507.638667,...,0.118718,0.881282,616.168067,570.414310,0.098721,0.098721,0.000365,0.999635,0.959044,0.959044
1,BENGALURU(R),0.742141,54895.937738,45017.109409,0.075418,0.075418,0.106882,0.893118,57992.196208,48927.510755,...,0.098063,0.901937,572.435052,534.632751,0.075418,0.075418,0.000279,0.999721,0.976505,0.976505
2,RAMANAGARA,0.600687,21642.413549,17809.680527,0.007616,0.007616,0.041388,0.958612,27479.191334,23962.324950,...,0.037966,0.962034,445.187197,430.520870,0.007616,0.007616,0.000028,0.999972,0.822250,0.822250
3,CHITRADURGA,0.419056,101798.443858,83391.887143,0.171050,0.171050,0.199260,0.800740,101029.330849,84139.711825,...,0.182827,0.817173,751.912067,681.477582,0.171050,0.171050,0.000632,0.999368,0.668650,0.668650
4,DAVANAGERE,0.106020,281678.371090,230566.373060,0.537816,0.537816,0.553544,0.446456,266084.831886,219185.121765,...,0.507914,0.492086,1440.240158,1244.655111,0.537816,0.537816,0.001988,0.998012,0.790058,0.790058


In [ ]:
# List of districts in the dataset
districts = df['District'].tolist()

# List of columns to visualize
columns_to_visualize = ['Rice_Production_223', 'Jowar_Yield_278', 'Maize_Production_226', 'SowingSeedsDistributed_Jowar_318', 'SowingSeedsDistributed_Maize_320']

colors = Category20[len(columns_to_visualize)]

# Create a figure for each column
for column, color in zip(columns_to_visualize, colors):
    # Create a new plot
    plot = figure(title=f'{column} vs District', x_range=districts, sizing_mode='stretch_width', toolbar_location=None, tools='')

    # Plot the data
    source = ColumnDataSource(data=dict(x=districts, y=df[column]))
    plot.vbar(x='x', top='y', width=0.8, color=color, legend_label=column, source=source)

    # Add hover tooltips
    hover = HoverTool()
    hover.tooltips = [('District', '@x'), (column, '@y')]
    plot.add_tools(hover)

    # Styling
    plot.xaxis.major_label_orientation = "vertical"
    plot.yaxis.axis_label = column
    plot.legend.location = 'top_right'
    plot.legend.click_policy = 'hide'

    # Show the plot
    show(plot)


In [ ]:
from bokeh.models import FactorRange
from bokeh.transform import factor_cmap

# Assuming df is your DataFrame

# List of crops
crops = ['Rice_Production_223', 'Jowar_Yield_278', 'Maize_Production_226']

# Create a new plot
plot = figure(title='Production Comparison by Crop and District', x_range=FactorRange(*districts),toolbar_location=None, tools='')

# Plot the data
source = ColumnDataSource(df)
plot.vbar_stack(crops, x='District', width=0.9, color=Category20[len(crops)], source=source, legend_label=crops)

# Add hover tooltips
hover = HoverTool()
hover.tooltips = [('District', '@District')] + [(crop, f'@{crop}') for crop in crops]
plot.add_tools(hover)

# Styling
plot.xaxis.major_label_orientation = "vertical"
plot.yaxis.axis_label = "Production"
plot.legend.location = 'top_left'
plot.legend.click_policy = 'hide'

# Show the plot
show(plot)
